# Plot Dot Plot

This notebook will help you generate "Prism-style" dot plots in Python, inspect the distribution of your data and run two-sample statistics.
<hr>

## Setup
First, we need to install a working version of Pandas. Run the cell below. You may need to restart the kernel after running it.

In [ ]:
%pip install -U pandas==1.5.3

## Import Data

> **Task**: 
> 1. Change `data_1` and `data_2` to be your two group of data. *Make sure you leave these as lists, with brackets on each end, and each data point separated by a comma.*
> 2. *Optional*: Rename `Condition_1` and `Condition_2`. *Make sure you keep these in single quotes, so Python recognizes them as a string!*

In [ ]:
# Option 1: Import a CSV file as a Pandas dataframe
import pandas as pd
#filename = ...
#data = pd.read_csv(filename)

# Option 2: Import your data as two lists and generate a dataframe from it
data_1 = [1,3,3,2,1,2,4,2,5]
data_2 = [3,4,5,3,2,6,7,8,3]
data = pd.DataFrame(data={'Condition_1':data_1,'Condition_2':data_2})
data

## Plot Data
Below, we'll use a seaborn plotting function called [swarmplot](https://seaborn.pydata.org/generated/seaborn.swarmplot.html) to plot each of our data points. 

### Notes 
* This will draw a <font color='gray'>**dotted gray**</font> line for the median, and a **solid black line** for the mean. 
* Change the `plt.ylabel` line to add your own label.

In [ ]:
# Import needed packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

# Set up the plot
fig,ax = plt.subplots(1,1,figsize=(4,4))

# plot the mean line
sns.boxplot(data=data, showmeans=True,meanline=True,
            meanprops={'color': 'gray', 'ls': '--', 'lw': 2},
            medianprops={'visible': True,'color': 'black', 'ls': '-', 'lw': 2},
            whiskerprops={'visible': False},
            showfliers=False,showbox=False,showcaps=False)

# plot individual data points
sns.swarmplot(data=data,s=8)

plt.ylabel('Thing we\'re measuring')

# Make the axes look nice!
ax.spines[['right', 'top']].set_visible(False)

plt.show()

## Check to see how skewed the data is

Before we run any hypoothesis tests, we need to know if our data is skewed or not. To test for skewness, we can use [`stats.skewtest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skewtest.html#scipy.stats.skewtest) to test. This method implements the <a href="https://www.jstor.org/stable/2684359?seq=1">D'Agostino-Pearson skewness test</a>, one of many different tests (e.g., the Kolmogorov-Smirov test) that can be used to check the normality of a distribution. **If the skew test gives us a p-value of less than 0.05, the population is skewed.**

>**Task**: Run the cell below, but then change the `sample` to `data_2` to test your second group of data points.

In [ ]:
from scipy import stats

sample = data_1 # Choose which data to use

stat,pvalue = stats.skewtest(sample) # Run the skew test

# Print the p value of the skew test up to 30 decimal points
print('The skewtest p-value is ' + '%.30f' % pvalue) 

plt.hist(sample) # Create a histogram
plt.ylabel('Observations')
plt.xlabel('Thing we\'re measuring')
plt.show()

## Run one-sample statistics (earthworm data)

### *Inferential statistics* generalize from observed data to the world at large
Most often, the goal of our hypothesis testing is to test whether or not two distributions are different, or if a distribution has a different mean than the underlying population distribution.

The code below will run a one sample t-test where individual samples are not being subtracted from each other. It will also calculate Cohen's *d*.

In [ ]:
import numpy as np

#get degrees of freedom and n
n1=len(data['Condition_1'])
n2=len(data['Condition_2'])
df=(n1)-1 #number of logically independent values

#get group1-group2 average 
avgdif=(np.mean(data['Condition_1'])-np.mean(data['Condition_2']))

# get combine stdev
var1=np.var(data['Condition_1'])
var2=np.var(data['Condition_2'])
covar=(np.cov(data['Condition_1'],data['Condition_2']))
stdev=np.sqrt((var1+var2)-(2*covar[0,1]))
std_error=stdev/np.sqrt(n1+n2)
print(var1)
print(var2)

t = abs(avgdif - 0) / std_error # get t statistic
p = 2*(1-stats.t.cdf(t,df)) # get two-tailed p value
print(p)

#If Variances are = or similar
#s = np.sqrt((((n1 - 1)*var1) + (((n2 - 1))*var2)) / (n1 + n2 - 2))
#cohen_d=avgdif/s

#If Variances are different
s=np.sqrt((var1/n1) + (var2/n2))
cohen_d=avgdif/s
print(cohen_d)

## Run two-sample statistics


The SciPy stats package has [many hypothesis testing tools](https://docs.scipy.org/doc/scipy/reference/stats.html) (see Statistical Tests). In most cases, we'd like to test whether two or more distributions are different from eachother.

If we know our distributions are normal (they're generated from a normal distribution!) we can use **parametric statistics** to test our hypothesis. To test for differences between normal populations, we can use the independent t-test in our stats package: `stats.ttest_ind()`. If we had paired samples, we would use a dependent t-test [as seen here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html#scipy.stats.ttest_rel).

If one of our populations is skewed, however, we **cannot use a t-test**. A t-test assumes that the populations are normally distributed. For skewed populations, we can use either the [Mann-Whitney U](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html#scipy.stats.mannwhitneyu) (for independent samples, `stats.mannwhitneyu()`) or the [Wilcoxon Signed Rank Test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html#scipy.stats.wilcoxon) (for dependent/paired samples,`stats.wilcoxon()`).

Below, there is sample code to run three different statistical tests. You should uncomment (remove the #) from the one that is most appropriate for your data. 

In [ ]:
#print(stats.ttest_ind(data_1,data_2)) # to run an independent t-test
#print(stats.mannwhitneyu(data_1,_2)) # to run a mannwhitneyu
#print(stats.wilcoxon(data_1,data_2)) # to run a wilcoxon signed rank test